In [0]:
%sql
CREATE CATALOG IF NOT EXISTS medalhao;
USE CATALOG medalhao;

CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA bronze;

CREATE VOLUME IF NOT EXISTS landing;

In [0]:
import requests
import pandas as pd
from pyspark.sql import SparkSession, functions as F

# Definição do caminho do schema Bronze
catalogo = 'medalhao'
bronze_db_name = 'bronze'

In [0]:
def ingest_csv(nome_arquivo, nome_tabela):
   
    try:
        landing_path = f'/Volumes/medalhao/bronze/landing/{nome_arquivo}'

        # Leitura do arquivo CSV
        df = spark.read.csv(landing_path, header=True, inferSchema=True)

        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f'O arquivo {nome_arquivo} está vazio ou não pôde ser lido.')

        # Adiciona timestamp de ingestão
        df_with_metadata = df.withColumn('data_ingestao', F.current_timestamp())

        # Escrita no formato Delta
        df_with_metadata.write.format('delta').mode('overwrite').saveAsTable(f'{catalogo}.{bronze_db_name}.{nome_tabela}')

        print(f'Tabela bronze.{nome_tabela} criada com sucesso!\n')

    except Exception as e:
        print(f'Erro ao processar {nome_tabela}: {str(e)}')

In [0]:
# criando tabelas no schema bronze

nomes_arquivos = [
    'olist_customers_dataset.csv',
    'olist_geolocation_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset.csv',
    'olist_orders_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv',
    'product_category_name_translation.csv'
]

nomes_tabelas = [
    'ft_consumidores',
    'ft_geolocalizacao',
    'ft_itens_pedidos',
    'ft_pagamentos_pedidos',
    'ft_avaliacoes_pedidos',
    'ft_pedidos',
    'ft_produtos',
    'ft_vendedores',
    'dm_categoria_produtos_traducao'
]

for i in range(0, len(nomes_arquivos)):
    ingest_csv(nomes_arquivos[i], nomes_tabelas[i])

In [0]:
# extraindo a cotacao do dolar via API

spark = SparkSession.builder.getOrCreate()

# formato MM-DD-AA
data_inicio_formatada = '01-01-2016'
data_fim_formatada = '01-01-2019'

nome_tabela = 'dm_cotacao_dolar'
url = (
f'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial=%27{data_inicio_formatada}%27&@dataFinalCotacao=%27{data_fim_formatada}%27&$select=dataHoraCotacao,cotacaoCompra&$format=json'
)

# get response
response = requests.get(url)

if response.status_code == 200:
    dados_json = response.json().get('value', [])

    if not dados_json:
        raise ValueError('Nenhum dado retornado pela API PTAX para o periodo informado.')

    df_pandas = pd.DataFrame(dados_json)
    df_spark = spark.createDataFrame(df_pandas)

    # adiciona o timestamp de ingestao
    df_spark = (
        df_spark.withColumn('data_ingestao', F.current_timestamp())
    )

    df_spark.write.format('delta').mode('overwrite').saveAsTable(
        f'{catalogo}.{bronze_db_name}.{nome_tabela}'
    )

    print(f'Dados gravados com sucesso em {catalogo}.{bronze_db_name}.{nome_tabela}')
else:
    print(f'Erro na requisicao: {response.status_code}')
    print(response.text)

In [0]:
# value da API
print(dados_json[0])

In [0]:
# catalog.bronze.landing.dm_cotacao_dolar
df_spark.limit(5).display()